In [1]:
import torch
import numpy as np
from torch import nn
import torch.autograd.profiler as profiler

In [2]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        
    def forward(self, input, mask):
        with profiler.record_function("linear pass"):
            out = self.linear(input)
        
        with profiler.record_function("mask indices"):
            threshold = out.sum(axis=1).mean().item()
            hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            hi_idx = torch.from_numpy(hi_idx).cuda()
        
        return out, hi_idx

In [5]:
model = MyModule(500, 10)